In [12]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import classification_report, confusion_matrix

# 0. Load datasets

In [2]:
datasets_path = lambda  file_name: f'/Users/stijnvanleeuwen/Desktop/codes/EUR/Ass2/datasets/{file_name}.parquet'

baskets = pd.read_parquet(datasets_path('baskets')).astype({'week':'uint8', 'customer':'category','product':'category', 'price':'uint16'}) 
coupons = pd.read_parquet(datasets_path('coupons')).astype({'week':'uint8', 'customer':'category','product':'category', 'discount':'uint8'})
prediction_index = pd.read_parquet(datasets_path('prediction_index')).astype({'week':'uint8', 'customer':'category','product':'category'}) 

# 1. Create base dataframe 

In [3]:
weeks = range(80,89+1)
customers = range(2000)
products = range(250)

n_weeks, n_customers, n_products = len(weeks), len(customers), len(products)

base = pd.DataFrame()

base['week'] =     np.array([[x] * n_products * n_customers for x in weeks]).flatten()
base['customer'] = np.array([[x] * n_products * n_weeks for x in customers]).flatten()
base['product'] =  np.array([[x] * n_weeks * n_customers for x in products]).flatten()

base = pd.merge(base, baskets, on=['week', 'customer','product'], how='left')

base['price'] = base['price'].fillna(0).astype('uint16')
base['bought'] = (base['price']!=0).astype(int)

In [5]:
base

,week,customer,product,price,bought
0,80,0,0,0,0
1,80,0,0,0,0
2,80,0,0,0,0
3,80,0,0,0,0
4,80,0,0,0,0
...,...,...,...,...,...
4999995,89,1999,249,0,0
4999996,89,1999,249,0,0
4999997,89,1999,249,0,0
4999998,89,1999,249,0,0


# 2. Split into train & test

In [8]:
train = base[base['week']!=89]
test =  base[base['week']==89]

x_train, x_test = train.drop(['price','bought'],axis=1), test.drop(['price','bought'],axis=1)
y_train, y_test = train['bought'], test['bought']

# 3. Random Forest Classification

In [14]:
# a. Train model
model = RandomForestClassifier()
model.fit(x_train, y_train)

In [ ]:
# b. Predict y on test
y_pred = model.predict(x_test)

In [ ]:
# c. Get confusion matrix
print(classification_report(y_test, y_pred))